## Transform MachineInfo from RDD to DataFrames

### TL;DR:

`Parqet` seems to reduce storage cost and `DataFrames` allows to make complex queries.

### Motivation
Expose information from MachineInfo as Spark’s DataFrames to provide SQL-based offline or on-demand analysis capabilities.

### Overview
* Extract and flatten MachineInfo to a couple of tables such as Processes, Events, Workloads, etc. Save the table in column-oriented `parquet` format to reduce the required disk space and hence improve the retention of these tables.
* Explore other ways to reduce the disk usage.
* Propose a couple of real-life usages of these DataFrames for Threat Hunting using Spark’s built-in aggregation functions and plugged-in user-defined functions.

### Future work
* Convert experimental code into `PsPipline` codebase
* Write tests


### Open question: 
    * What if we would like to extract a part of RDD. Can we have a "loose" schema to extract only needed data?

### 1. Import  dependencies

* Assume that `PsPipeline` has been compiled into a jar file.


In [1]:
// Ref: https://toree.apache.org/docs/current/user/faq/
%AddJar file:///Users/datng2/pspipline.jar
sc.setLogLevel("DEBUG")

Starting download from file:///Users/datng2/pspipline.jar
Finished download of pspipline.jar


In [2]:
// For implicit conversions from RDDs to DataFrames
// https://stackoverflow.com/questions/44094108/not-able-to-import-spark-implicits-in-scalatest
val spark2 = spark
val sqlContext= new org.apache.spark.sql.SQLContext(sc)
import spark2.implicits._
import sqlContext.implicits._

import java.net.URI
import collection.JavaConverters._
import org.apache.spark.sql.Row
import org.apache.hadoop.fs.{FileSystem, Path}
import com.tetration.proto.MachineInfoProtos.MachineInfo
import com.tetration.processanalytics.pipeline.io.MachineInfoBatchReaderWithProtoField

spark2 = org.apache.spark.sql.SparkSession@6e60574d
sqlContext = org.apache.spark.sql.SQLContext@107986c4


org.apache.spark.sql.SQLContext@107986c4

### 2. Load `RDD[MachineInfo]`

In [3]:
import java.net.URI
import org.apache.hadoop.fs.{FileSystem, Path}

final val DATA_DIR = "/Users/datng2/data/machineinfo/galois"
final val NUM_BATCHES = 5

// Print all files in a directory
val batches = FileSystem
    .get(URI.create("file:///"), sc.hadoopConfiguration)
    .listStatus(new Path(DATA_DIR))
    .map(_.getPath.toString.split("/").last)  // extract filename from an absolute path
    .take(NUM_BATCHES)  // select first N-items 
    .toSeq    //convert to seq

val miProtoReader = new MachineInfoBatchReaderWithProtoField(
    sc,
    sc.hadoopConfiguration,
    FileSystem.get(URI.create("file:///"), sc.hadoopConfiguration),
    DATA_DIR)

val miRDDs = miProtoReader
    .readBatches(batches.asJava).asScala
    .reduce(_ union _)
    .rdd

DATA_DIR = /Users/datng2/data/machineinfo/galois
NUM_BATCHES = 5
batches = WrappedArray(201805091922, 201805092101, 201805091925, 201805091913, 201805091914)
miProtoReader = com.tetration.processanalytics.pipeline.io.MachineInfoBatchReaderWithProtoField@6f439199
miRDDs = UnionRDD[23] at union at <console>:77


UnionRDD[23] at union at <console>:77

In [37]:
val test = miRDDs.first

test = 


host_name: "collectorDatamover-2"
sensor_id: "fda22d272815f15031cb91af50df1fa7621c310c"
process_info {
  process_id: 2396
  user_id: "collectd"
  parent_process_id: 24429
  command_string: "ping\000-c2\00010.28.1.193\000"
  start_time_usec: 1525893688000000
  end_time_usec: 1525893689637822
  exec_path: "/bin/ping"
  file_attr {
    size: 40760
    ctime: 1486575379552999998
    mtime: 1380206111000000000
    mode: 35309
  }
  thread_group_id: 2396
  parent_thread_group_id: 24429
  exec_time_usec: 1525893688634884
  privilege_escalated: true
  exit_code: 0
  exit_signal: 17
  username: "collectd"
  uid: 513
  uptime_usec: 1000000
  forensic_events {
    security_event: PRIV_ESCALATION
    priv_escalation_data {
      types_bitmap...


In [42]:
test.getProcessInfoList.get(0).getMemoryUsageKB

106716

### 3. Define  an experimental schema

* The idea is to convert **RDD[MachineInfo] into RDD[Row]**, where each Row contains the information that our pipeline requires. In Spark 2.0, DataFrame is a type alias for *Dataset of Row*.
* We could potentially consider leveraging `protobuf` definition to transform RDD to DataFrame. However, we currently use a simplified schema for testing purpose.

In [36]:
case class ProcessInfoRow(
    val sensorId: String,
    val processKeyPart1: Long,
    val processKeyPart2: Long,
    val cpuUsage: Long,
    val memoryUsage: Long,
    val commandString: String
)

defined class ProcessInfoRow


In [44]:
// Transform functions
val MIToProcessInfo: MachineInfo => List[ProcessInfoRow] = mi => {
  mi.getProcessInfoList.asScala
    .map(pi => ProcessInfoRow(
        mi.getSensorId,
        pi.getKey.getPart1,
        pi.getKey.getPart2,
        pi.getCpuUsageUs,
        pi.getMemoryUsageKB,
        pi.getCommandString))
    .toList
}

val processInfoDF = miRDDs
    .flatMap(MIToProcessInfo)
    .toDF

MIToProcessInfo = > List[ProcessInfoRow] = <function1>
processInfoDF = [sensorId: string, processKeyPart1: bigint ... 4 more fields]


[sensorId: string, processKeyPart1: bigint ... 4 more fields]

### 4. Sample Queries:
Ref: https://github.com/apache/incubator-toree/blob/master/etc/examples/notebooks/magic-tutorial.ipynb

In [46]:
// Create a ProcessInfo Table
processInfoDF.createOrReplaceTempView("ProcessInfo")

// Count number of rows sensorId
sqlContext
    .sql("""SELECT COUNT(*) FROM ProcessInfo""").show

+--------+
|count(1)|
+--------+
|    3488|
+--------+



In [47]:
// Create a SQL query to count number of unique command strings in each sensorId
sqlContext
    .sql("""
        SELECT sensorId, COUNT(DISTINCT commandString) FROM ProcessInfo
            GROUP BY sensorId
    """)
    .show(10)

+--------------------+-----------------------------+
|            sensorId|count(DISTINCT commandString)|
+--------------------+-----------------------------+
|fda22d272815f1503...|                          303|
|7f3fc376d08f771af...|                          431|
|11f991d882bdab1f7...|                          307|
|20bee197e8368b81b...|                          315|
|a26526ab2b11a0a1c...|                          318|
|967d3582a6a9bc0da...|                          398|
+--------------------+-----------------------------+



In [48]:
// Compute average CPU usage (in microsecond)
sqlContext
    .sql("""
        SELECT AVG(cpuUsage) FROM ProcessInfo
    """).show

+--------------------+
|       avg(cpuUsage)|
+--------------------+
|3.1103188016055045E9|
+--------------------+



### 5. Saving to Parquet

In [12]:
sqlContext.setConf("spark.sql.parquet.compression.codec", "gzip")
processInfoDF.write.parquet("pi.parquet")

import org.apache.hadoop.io.compress.GzipCodec
processInfoDF.rdd.saveAsTextFile("pi.rdd", classOf[GzipCodec])

lastException: Throwable = null


In [35]:
val fs = FileSystem.get(URI.create("file:///"), sc.hadoopConfiguration)

val path = "/Users/datng2/internship/mi-spark/"

println("Rdd     size: " + fs.listStatus(new Path(path + "pi.rdd")).map(x => x.getBlockSize()).sum / 1024 + " kB")
println("Parquet size: " + fs.listStatus(new Path(path + "pi.parquet")).map(x => x.getBlockSize()).sum / 1024 + " kB")

Rdd     size: 3964928 kB
Parquet size: 786432 kB


fs = org.apache.hadoop.fs.LocalFileSystem@447a4fea
path = /Users/datng2/internship/mi-spark/


/Users/datng2/internship/mi-spark/